### ДЗ2: Группировка данных и оконные функции
Будем использовать те же таблицы, что и в ДЗ2.

In [1]:
import psycopg2
import pandas as pd

In [ ]:
host = "localhost" 
dbname = "postgres"
user = "postgres" 
password = "" 
port = 5432 

connection = psycopg2.connect(
        host=host,
        dbname=dbname,
        user=user,
        password=password,
        port=port
)

#### 1. Вывести распределение (количество) клиентов по сферам деятельности, отсортировав результат по убыванию количества.

In [3]:
cursor = connection.cursor()

cursor.execute("""
    SELECT job_industry_category,
            COUNT(customer_id) as counts
    FROM customer
    GROUP BY job_industry_category
    ORDER BY counts DESC;
""")

display(pd.DataFrame(cursor.fetchall()))

,0,1
0,Manufacturing,799
1,Financial Services,774
2,n/a,656
3,Health,602
4,Retail,358
5,Property,267
6,IT,223
7,Entertainment,136
8,Argiculture,113
9,Telecommunications,72


#### 2. Найти сумму транзакций за каждый месяц по сферам деятельности, отсортировав по месяцам и по сфере деятельности.

In [13]:
cursor.execute("""
    SELECT TO_CHAR(transaction_date, 'YYYY-MM') as dt,
            job_industry_category,
            SUM(list_price)
    FROM transaction AS t
    JOIN customer AS c USING(customer_id)
    GROUP BY dt, job_industry_category
    ORDER BY dt, job_industry_category;
""")

display(pd.DataFrame(cursor.fetchall()))

,0,1,2
0,2017-01,Argiculture,43513.82
1,2017-01,Entertainment,64089.92
2,2017-01,Financial Services,366383.71
3,2017-01,Health,286860.38
4,2017-01,IT,107783.37
...,...,...,...
115,2017-12,Manufacturing,319248.23
116,2017-12,n/a,300793.18
117,2017-12,Property,117263.20
118,2017-12,Retail,153369.72


#### 3. Вывести количество онлайн-заказов для всех брендов в рамках подтвержденных заказов клиентов из сферы IT.

In [4]:
cursor.execute("""
    SELECT COUNT(transaction_id)
    FROM transaction AS t
    JOIN customer AS c USING(customer_id)
    WHERE t.online_order = TRUE 
        AND c.job_industry_category = 'IT'
            AND t.order_status = 'Approved';
""")

display(pd.DataFrame(cursor.fetchall()))

,0
0,540


#### 4. Найти по всем клиентам сумму всех транзакций (list_price), максимум, минимум и количество транзакций, отсортировав результат по убыванию суммы транзакций и количества клиентов. Выполните двумя способами: используя только group by и используя только оконные функции. Сравните результат. 

В задании написано что-то странное: в данном случае отсортировать я могу по количеству транзакций, но никак не по количеству клиентов.

In [15]:
cursor.execute("""
    SELECT t.customer_id,
            SUM(list_price) as sm,
            MAX(list_price),
            MIN(list_price),
            COUNT(transaction_id) as cnt
    FROM transaction AS t
    JOIN customer AS c USING(customer_id)
    GROUP BY t.customer_id
    ORDER BY sm DESC, cnt DESC;
""")

display(pd.DataFrame(cursor.fetchall()))

,0,1,2,3,4
0,2183,19071.32,2005.66,230.91,14
1,1129,18349.27,1992.93,290.62,13
2,1597,18052.68,2091.47,360.40,12
3,941,17898.46,2091.47,1057.51,10
4,2788,17258.94,2083.94,183.86,11
...,...,...,...,...,...
3488,2423,202.62,202.62,202.62,1
3489,3189,200.70,100.35,100.35,2
3490,2274,142.98,71.49,71.49,2
3491,2532,71.49,71.49,71.49,1


Теперь то же самое, но с оконной функцией и без группировки.

In [16]:
cursor.execute("""
    SELECT DISTINCT
        t.customer_id,
        SUM(t.list_price) OVER (PARTITION BY t.customer_id) AS sm,
        MAX(t.list_price) OVER (PARTITION BY t.customer_id),
        MIN(t.list_price) OVER (PARTITION BY t.customer_id),
        COUNT(t.transaction_id) OVER (PARTITION BY t.customer_id) AS cnt
    FROM transaction AS t
    JOIN customer AS c USING(customer_id)
    ORDER BY sm DESC, cnt DESC;
""")

display(pd.DataFrame(cursor.fetchall()))

,0,1,2,3,4
0,2183,19071.32,2005.66,230.91,14
1,1129,18349.27,1992.93,290.62,13
2,1597,18052.68,2091.47,360.40,12
3,941,17898.46,2091.47,1057.51,10
4,2788,17258.94,2083.94,183.86,11
...,...,...,...,...,...
3488,2423,202.62,202.62,202.62,1
3489,3189,200.70,100.35,100.35,2
3490,2274,142.98,71.49,71.49,2
3491,2532,71.49,71.49,71.49,1


Сравнивая эти два запроса, можно отметить, что запрос с GROUP BY короче и более читаем. Для несложных задач аггрегации, где нет необходимости сохранять детализацию, он будет более предпочтительным.

#### 5. Найти имена и фамилии клиентов с минимальной/максимальной суммой транзакций за весь период (сумма транзакций не может быть null). Напишите отдельные запросы для минимальной и максимальной суммы.

In [ ]:
cursor.execute("""
    WITH s1 AS (
        SELECT c.customer_id,
                c.first_name,
                c.last_name,
                SUM(t.list_price) AS total_spent
        FROM customer AS c
        JOIN transaction AS t USING(customer_id)
        GROUP BY c.customer_id, c.first_name, c.last_name
    )
    SELECT customer_id,
            first_name,
            last_name,
            total_spent
        FROM s1
        WHERE total_spent = (SELECT MIN(total_spent) FROM s1);  
""")

display(pd.DataFrame(cursor.fetchall()))

,0,1,2,3
0,3292,Hamlen,Slograve,60.34


In [23]:
cursor.execute("""
    WITH s1 AS (
        SELECT c.customer_id,
                c.first_name,
                c.last_name,
                SUM(t.list_price) AS total_spent
        FROM customer AS c
        JOIN transaction AS t USING(customer_id)
        GROUP BY c.customer_id, c.first_name, c.last_name
    )
    SELECT customer_id,
            first_name,
            last_name,
            total_spent
    FROM s1
    WHERE total_spent = (SELECT MAX(total_spent) FROM s1);  
""")

display(pd.DataFrame(cursor.fetchall()))

,0,1,2,3
0,2183,Jillie,Fyndon,19071.32


#### 6. Вывести только самые первые транзакции клиентов. Решить с помощью оконных функций.

In [ ]:
cursor.execute("""
    SELECT s1.transaction_id
    FROM (
        SELECT c.customer_id,
                t.transaction_id,
                ROW_NUMBER() OVER(PARTITION BY c.customer_id ORDER BY transaction_date) as t_number
        FROM customer AS c
        JOIN transaction AS t USING(customer_id)
    ) AS s1
    WHERE s1.t_number = 1;
""")

display(pd.DataFrame(cursor.fetchall()))

,0
0,9785
1,2261
2,10302
3,12441
4,2291
...,...
3488,9769
3489,8276
3490,13469
3491,2794


#### 7. Вывести имена, фамилии и профессии клиентов, между транзакциями которых был максимальный интервал (интервал вычисляется в днях)

In [50]:
cursor.execute("""
    WITH s1 AS(
        SELECT c.customer_id,
                c.first_name,
                c.last_name,
                c.job_title,
                t.transaction_date as dt,
                COALESCE((
                    t.transaction_date - LAG(t.transaction_date) OVER (
                        PARTITION BY c.customer_id ORDER BY t.transaction_date)
                    ), 0
                ) AS interval_days
        FROM customer AS c
        JOIN transaction AS t USING(customer_id)
        ORDER BY c.customer_id, dt
    )
    SELECT s1.first_name,
            s1.last_name,
            s1.job_title,
            s1.interval_days
    FROM s1
    ORDER BY s1.interval_days DESC
    LIMIT 10;
""")

display(pd.DataFrame(cursor.fetchall()))

,0,1,2,3
0,Susanetta,,Legal Assistant,357
1,Royall,Terris,Geological Engineer,330
2,Gregorius,Cockram,Data Coordiator,330
3,Stoddard,Giacomoni,Structural Analysis Engineer,330
4,Bearnard,Letixier,,329
5,Caralie,Sellors,Senior Editor,321
6,Debee,Martynov,Senior Editor,320
7,Genni,Larway,Environmental Specialist,314
8,Timmie,Lenden,,310
9,Carolynn,Samsin,Pharmacist,310


In [28]:
cursor.execute("ROLLBACK")